Lambda School Data Science

*Unit 2, Sprint 3, Module 2*

---


# Wrangle ML datasets 🍌


In today's lesson, we’ll work with a dataset of [3 Million Instacart Orders, Open Sourced](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2)!



### Setup

In [1]:
# Download data
import requests

def download(url):
    filename = url.split('/')[-1]
    print(f'Downloading {url}')
    r = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(r.content)
    print(f'Downloaded {filename}')

download('https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz')

Downloaded instacart_online_grocery_shopping_2017_05_01.tar.gz


In [2]:
# Uncompress data
import tarfile
tarfile.open('instacart_online_grocery_shopping_2017_05_01.tar.gz').extractall()

In [3]:
# Change directory to where the data was uncompressed
%cd instacart_2017_05_01

/Users/loris/Documents/Lambda School/DS-Unit-2-Applied-Modeling/module2/instacart_2017_05_01


In [4]:
# Print the csv filenames
from glob import glob
for filename in glob('*.csv'):
    print(filename)

products.csv
orders.csv
order_products__train.csv
departments.csv
aisles.csv
order_products__prior.csv


### For each csv file, look at its shape & head 

In [10]:
import pandas as pd
from IPython.display import display # to make df heads cleaner

def preview():
    for filename in glob('*.csv'):
        df = pd.read_csv(filename)
        print(filename, df.shape)
        display(df.head())
        print('\n')
        
preview()

products.csv (49688, 4)


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13




orders.csv (3421083, 7)


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0




order_products__train.csv (1384617, 4)


...

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol




aisles.csv (134, 2)


,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation




order_products__prior.csv (32434489, 4)


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


## The original task was complex ...

[The Kaggle competition said,](https://www.kaggle.com/c/instacart-market-basket-analysis/data):

> The dataset for this competition is a relational set of files describing customers' orders over time. The goal of the competition is to predict which products will be in a user's next order.

> orders.csv: This file tells to which set (prior, train, test) an order belongs. You are predicting reordered items only for the test set orders.

Each row in the submission is an order_id from the test set, followed by product_id(s) predicted to be reordered.

> sample_submission.csv: 
```
order_id,products
17,39276 29259
34,39276 29259
137,39276 29259
182,39276 29259
257,39276 29259
```

## ... but we can simplify!

Simplify the question, from "Which products will be reordered?" (Multi-class, [multi-label](https://en.wikipedia.org/wiki/Multi-label_classification) classification) to **"Will customers reorder this one product?"** (Binary classification)

Which product? How about **the most frequently ordered product?**

# Questions:

- What is the most frequently ordered product?
- How often is this product included in a customer's next order?
- Which customers have ordered this product before?
- How can we get a subset of data, just for these customers?
- What features can we engineer? We want to predict, will these customers reorder this product on their next order?

## What was the most frequently ordered product?

In [14]:
prior = pd.read_csv('order_products__prior.csv')

In [13]:
prior['product_id'].mode()

0    24852
dtype: int64

In [15]:
prior['product_id'].value_counts()

24852    472565
13176    379450
21137    264683
21903    241921
47209    213584
          ...  
11356         1
18001         1
6320          1
26268         1
30087         1
Name: product_id, Length: 49677, dtype: int64

In [16]:
train = pd.read_csv('order_products__train.csv')
train['product_id'].mode()

0    24852
dtype: int64

In [17]:
train['product_id'].value_counts()

24852    18726
13176    15480
21137    10894
21903     9784
47626     8135
         ...  
44256        1
2764         1
4815         1
43736        1
46835        1
Name: product_id, Length: 39123, dtype: int64

In [18]:
# what is that product?
products = pd.read_csv('products.csv')

In [19]:
products[products['product_id']==24852]

,product_id,product_name,aisle_id,department_id
24851,24852,Banana,24,4


In [20]:
# or we can merge the dataframes
prior = pd.merge(prior, products, on='product_id')

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,2,33120,1,1,Organic Egg Whites,86,16
1,26,33120,5,0,Organic Egg Whites,86,16
2,120,33120,13,0,Organic Egg Whites,86,16
3,327,33120,5,1,Organic Egg Whites,86,16
4,390,33120,28,1,Organic Egg Whites,86,16
...,...,...,...,...,...,...,...
32434484,3265099,43492,3,0,Gourmet Burger Seasoning,104,13
32434485,3361945,43492,19,0,Gourmet Burger Seasoning,104,13
32434486,3267201,33097,2,0,Piquillo & Jalapeno Bruschetta,81,15
32434487,3393151,38977,32,0,Original Jerky,100,21


## How often is this product (bananas) included in a customer's next order?

There are [three sets of data](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b):

> "prior": orders prior to that users most recent order (3.2m orders)  
"train": training data supplied to participants (131k orders)  
"test": test data reserved for machine learning competitions (75k orders)

Customers' next orders are in the "train" and "test" sets. (The "prior" set has the orders prior to the most recent orders.)

We can't use the "test" set here, because we don't have its labels (only Kaggle & Instacart have them), so we don't know what products were bought in the "test" set orders.

So, we'll use the "train" set. It currently has one row per product_id and multiple rows per order_id.

But we don't want that. Instead we want one row per order_id, with a binary column: "Did the order include the product?"

Let's wrangle!

In [31]:
# how do we turn this into a df that has just one row / order & has a column t/f for banana presence
df = train.head(16).copy() # to ignore setting with copy warning

In [33]:
df['bananas'] = df['product_id'] == 24852
df

,order_id,product_id,add_to_cart_order,reordered,bananas
0,1,49302,1,1,False
1,1,11109,2,1,False
2,1,10246,3,0,False
3,1,49683,4,0,False
4,1,43633,5,1,False
5,1,13176,6,0,False
6,1,47209,7,0,False
7,1,22035,8,1,False
8,36,39612,1,0,False
9,36,19660,2,1,False


In [35]:
df.groupby('order_id')['bananas'].any()

order_id
1     False
36    False
Name: bananas, dtype: bool

In [43]:
train['bananas'] = train['product_id'] == 24852
train.groupby('order_id')['bananas'].any().reset_index()

,order_id,bananas
0,1,False
1,36,False
2,38,False
3,96,False
4,98,False
...,...,...
131204,3421049,False
131205,3421056,False
131206,3421058,False
131207,3421063,False


In [48]:
train_wrangled = train.groupby('order_id')['bananas'].any().reset_index()
# reset index to become df again

In [49]:
target = 'bananas'
train_wrangled[target].value_counts(normalize=True)

False    0.857281
True     0.142719
Name: bananas, dtype: float64

In [50]:
# Technique number 2
df.groupby('order_id')['product_id'].apply(list)

order_id
1     [49302, 11109, 10246, 49683, 43633, 13176, 472...
36    [39612, 19660, 49235, 43086, 46620, 34497, 486...
Name: product_id, dtype: object

In [57]:
# Group by order_id, get a list of product_ids for that order, check if that list includes bananas

def includes_bananas(product_ids):
    return 24852 in list(product_ids)

train_two = train.groupby('order_id')['product_id'].apply(includes_bananas)

In [58]:
train_two = (train_two
        .groupby('order_id')
        .agg({'product_id': includes_bananas})
        .reset_index()
        .rename(columns={'product_id': 'bananas'}))
target = 'bananas'
train[target].value_counts(normalize=True)

# Look this up ! 

/Users/loris/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  This is separate from the ipykernel package so we can avoid doing imports until


False    0.986476
True     0.013524
Name: bananas, dtype: float64

## Which customers have ordered this product before?

- Customers are identified by `user_id`
- Products are identified by `product_id`

Do we have a table with both these id's? (If not, how can we combine this information?)

In [ ]:
# No we don't have a table with both of these, but we have multiple with both 

In [61]:
# In the order_products__prior table, which orders included bananas?
BANANAS = 24852
prior[prior.product_id==BANANAS]

,order_id,product_id,add_to_cart_order,reordered
77,10,24852,1,1
180,20,24852,6,0
190,22,24852,3,1
234,26,24852,2,1
414,52,24852,2,1
...,...,...,...,...
32433984,3421027,24852,3,1
32434016,3421030,24852,9,1
32434146,3421038,24852,2,0
32434447,3421078,24852,2,1


In [62]:
banana_prior_order_ids = prior[prior.product_id==BANANAS].order_id

In [63]:
# Look at the orders table -- which orders included bananas?
orders = pd.read_csv('orders.csv')
orders.sample(n=5)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
1803873,1708052,108350,prior,9,4,17,8.0
656043,2273037,39500,prior,3,4,11,30.0
2385539,2460823,143550,prior,6,2,11,1.0
1358015,2150353,81625,prior,60,4,8,3.0
2560880,1143214,154109,prior,6,0,17,9.0


In [64]:
# In the orders table, which orders include bananas?
orders[orders.order_id.isin(banana_prior_order_ids)]

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
14,738281,2,prior,4,2,10,8.0
16,1199898,2,prior,6,2,9,13.0
17,3194192,2,prior,7,2,12,14.0
18,788338,2,prior,8,1,15,27.0
19,1718559,2,prior,9,2,9,8.0
...,...,...,...,...,...,...,...
3420915,1764570,206202,prior,20,4,0,11.0
3421078,2266710,206209,prior,10,5,18,29.0
3421079,1854736,206209,prior,11,4,10,30.0
3421080,626363,206209,prior,12,1,12,18.0


In [65]:
# check this order id, confirm that it includes bananas
prior[prior.order_id==738281]

,order_id,product_id,add_to_cart_order,reordered
6992648,738281,49451,1,0
6992649,738281,32792,2,1
6992650,738281,32139,3,0
6992651,738281,34688,4,0
6992652,738281,36735,5,0
6992653,738281,37646,6,0
6992654,738281,22829,7,0
6992655,738281,24852,8,0
6992656,738281,47209,9,0
6992657,738281,33276,10,0


In [66]:
banana_orders = orders[orders.order_id.isin(banana_prior_order_ids)]

In [67]:
# In the orders table, which users have bought bananas?
banana_user_ids = banana_orders.user_id.unique()

## How can we get a subset of data, just for these customers?

We want *all* the orders from customers who have *ever* bought the product.

(And *none* of the orders from customers who have *never* bought the product.)

In [75]:
# orders table, shape before getting subset
orders.shape

(3421083, 7)

In [77]:
# orders table, shape after getting subset
orders = orders[orders.user_id.isin(banana_user_ids)]
orders.shape
# this is now the subset of people who have ever bought bananas


(1512975, 7)

In [79]:
# IDs of all the orders from customers who have ever bought bananas
subset_order_ids = orders.order_id.unique()

In [80]:
# order_products__prior table, shape before getting subset
prior.shape

(32434489, 4)

In [81]:
# order_products__prior table, shape after getting subset
prior = prior[prior.order_id.isin(subset_order_ids)]
prior.shape

(16534534, 4)

In [82]:
# order_products__train table, shape before getting subset
train.shape

(1384617, 5)

In [84]:
# order_products__train table, shape after getting subset
train_wrangled = train_wrangled[train_wrangled.order_id.isin(subset_order_ids)]
train_wrangled.shape

(46964, 2)

In [85]:
# In this subset, how often were bananas reordered in the customer's most recent order?
train_wrangled[target].value_counts(normalize=True)

# this means that if you have ever bought bananas from instacart, you are more than 2x as 
# likely to order bananas again, as compared to the general population

False    0.647453
True     0.352547
Name: bananas, dtype: float64

## What features can we engineer? We want to predict, will these customers reorder this product on their next order?

In [87]:
train_wrangled.shape

(46964, 2)

In [89]:
train_wrangled.head()

,order_id,bananas
0,1,False
1,36,False
9,349,False
13,631,False
18,878,True


In [92]:
# merge user_id, order_ number, order_dow, order_hour_of_day, and days_since_prior_order
# with the training data
train = pd.merge(train_wrangled, orders)

In [93]:
train.head()
# How useful are each of these columns? 
# order_id could just be useful for merging other features
# bananas is the target
# user_id = use to merge features (maybe also length of customer time?)
# eval_set = constant
# last three columns are features

,order_id,bananas,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1,False,112108,train,4,4,10,9.0
1,36,False,79431,train,23,6,18,30.0
2,349,False,156353,train,9,3,16,30.0
3,631,False,184099,train,7,3,9,30.0
4,878,True,61911,train,9,2,13,30.0


In [ ]:
# Frequency of banana orders
# - % of orders
# - every n days on average

# Recency of banana orders 
# - n of orders
# - n days